In [1]:
import os, csv
import yfinance as yf
import pandas
def snapshot():
    with open('datasets/Book1.csv') as f:
        for line in f:
            if "," not in line:
                continue
            symbol = line.split(",")[1]
            Name = line.split(",")[0]
            #data = yf.download(symbol, start="2020-04-02", end="2021-11-6")
            data = yf.download(symbol, period="4mo", threads = True)
            data.to_csv('datasets/daily/{}.csv'.format(Name))
    return {
        "code": "success"
    }

snapshot()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

{'code': 'success'}

In [11]:
# valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
# Valid periods : 1d, 5d, 1mo, 3mo, 6mo, 1y, 2y, 5y, 10y, ytd, max
import yfinance as yf
tok = yf.download('tatamotors.ns', period="7d", interval='1m')
#data = yf.download('shoperstop.ns', period="1d",interval = "1m", threads = True)
#close = year['Close']    
#close[-200:].mean()
#close[-50:].mean()
#tok['200ma'] = tok['Adj Close'].rolling(window=200, min_periods=0).mean()
#tok['100ma'] = tok['Adj Close'].rolling(window=100, min_periods=0).mean()
tok

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2021-11-25 09:15:00+05:30,491.000000,492.950012,489.899994,490.799988,490.799988,0
2021-11-25 09:16:00+05:30,491.200012,494.000000,491.049988,493.700012,493.700012,326690
2021-11-25 09:17:00+05:30,493.850006,493.850006,492.600006,492.649994,492.649994,222374
2021-11-25 09:18:00+05:30,492.700012,494.000000,492.600006,493.799988,493.799988,200509
2021-11-25 09:19:00+05:30,493.850006,494.649994,493.750000,494.299988,494.299988,202488
...,...,...,...,...,...,...
2021-12-03 15:24:00+05:30,480.549988,480.549988,479.799988,479.799988,479.799988,117588
2021-12-03 15:25:00+05:30,479.950012,479.950012,479.750000,479.799988,479.799988,59554
2021-12-03 15:26:00+05:30,479.850006,480.000000,479.500000,479.799988,479.799988,97535


In [ ]:
import os, pandas
import pandas as pd
Consol = []
Break = []
def is_consolidating(df, percentage=2):
    recent_candlesticks = df[-10:]
    #print(percentage)
    max_close = recent_candlesticks['Close'].max()
    min_close = recent_candlesticks['Close'].min()

    threshold = 1 - (percentage / 100)
    if min_close > (max_close * threshold):
        return True        
    
    return False

def is_breaking_out(df, percentage=3.5):
    last_close = df[-1:]['Close'].values[0]

    if is_consolidating(df[:-1], percentage=percentage):
        recent_closes = df[-11:-1]

        if last_close > recent_closes['Close'].max():
            return True

    return False

for filename in os.listdir('datasets/daily'):
    df = pandas.read_csv('datasets/daily/{}'.format(filename))

    if is_consolidating(df, percentage=3.0):
        consol = filename + " is consolidating"
        Consol.append(consol)
    if is_breaking_out(df):
        break1 = filename + " is Breaking Out"
        Break.append(break1)
Consolidated = pd.DataFrame(Consol)
Breakout = pd.DataFrame(Break)

In [ ]:
import os, pandas
import pandas as pd
colab = []
def vol_break(df):
    recent_candlesticks = df[-15:-1]
    global Volume
    global last
    Volume = int(recent_candlesticks['Volume'].mean())
    last = df[-1:]['Volume'].values[0]
    
def breakout(Volume, last):
    if last>Volume:        
        return (Volume,last,filename)
    
for filename in os.listdir('datasets/daily'):
    df = pandas.read_csv('datasets/daily/{}'.format(filename))
     
    consol = vol_break(df)
    if breakout(Volume, last):
        colab.append(filename)
Breakout = pd.DataFrame(colab)

In [ ]:
Breakout

In [ ]:
import os, pandas
import pandas as pd
for filename in os.listdir('datasets/daily'):
    df = pandas.read_csv('datasets/daily/{}'.format(filename))

In [ ]:
pip install yfinance